In [109]:
print("hello world")
import numpy as np

hello world


In [110]:
import pandas as pd
import cv2
from PIL import Image
train = pd.read_csv("./images/train.csv")
X_train = train.drop(columns=['label'])
temp = np.array(X_train)
temp = temp.reshape(-1,28,28)
temp = Image.fromarray(temp[0].astype(np.uint8))
temp.save("test.png")
# cv2.imwrite("./test.png",temp)
exit

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)

Y_train = train['label']
Y_train = pd.get_dummies(train['label'], columns=['label'])
Y_train

,0,1,2,3,4,5,6,7,8,9
0,0,1,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
41995,1,0,0,0,0,0,0,0,0,0
41996,0,1,0,0,0,0,0,0,0,0
41997,0,0,0,0,0,0,0,1,0,0
41998,0,0,0,0,0,0,1,0,0,0


In [111]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
X_train, X_validation, Y_train, Y_validation = train_test_split(X_train,Y_train,test_size=0.1)

In [112]:
import numpy as np
X_train = np.array(X_train)
Y_train = np.array(Y_train)

In [113]:
X_train = X_train.reshape((-1,28,28,1))
X_train.shape

(37800, 28, 28, 1)

In [114]:
len(Y_train[0])

10

In [115]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

model = models.Sequential()
model.add(layers.Conv2D(16, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_19 (Conv2D)          (None, 26, 26, 16)        160       
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 13, 13, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_20 (Conv2D)          (None, 11, 11, 32)        4640      
                                                                 
 flatten_9 (Flatten)         (None, 3872)              0         
                                                                 
 dense_18 (Dense)            (None, 32)                123936    
                                                                 
 dense_19 (Dense)            (None, 10)                330       
                                                      

In [116]:
import tensorflow as tf
model.compile(optimizer="adam",loss="categorical_crossentropy", metrics=['accuracy'])
model.fit(x=X_train, y=Y_train, epochs=10)

Epoch 1/5
1182/1182 [==============================] - 6s 4ms/step - loss: 0.2128 - accuracy: 0.9370
Epoch 2/5
1182/1182 [==============================] - 5s 4ms/step - loss: 0.0707 - accuracy: 0.9781
Epoch 3/5
1182/1182 [==============================] - 5s 4ms/step - loss: 0.0495 - accuracy: 0.9840
Epoch 4/5
1182/1182 [==============================] - 5s 5ms/step - loss: 0.0359 - accuracy: 0.9889
Epoch 5/5
1182/1182 [==============================] - 5s 4ms/step - loss: 0.0260 - accuracy: 0.9914


In [117]:
X_validation = np.array(X_validation).reshape(-1,28,28,1)
result = model.predict(x=X_validation)
import typing
def getResult(raw_result:list):
    max_val = -1
    max_index = 0
    for index, val in enumerate(raw_result):
        if val > max_val:
            max_index = index
            max_val = val
    return max_index

result = list(map(lambda x: getResult(x),result))
print(result[0:20])
Y_validation = list(map(lambda x: getResult(x),np.array(Y_validation)))
print(Y_validation[0:20])

[5, 3, 6, 6, 2, 3, 2, 8, 4, 5, 0, 3, 1, 5, 4, 2, 8, 3, 6, 7]
[5, 3, 6, 6, 2, 3, 2, 8, 4, 3, 0, 3, 1, 5, 4, 2, 8, 3, 6, 7]


In [118]:
import cv2
img = cv2.imread('./4_test.png',cv2.IMREAD_GRAYSCALE)
output = cv2.resize(img, (28,28))
img = np.array(output)
img = np.array(list(map(lambda x: 255 - x, img.reshape(-1,1))))
print(img.shape)
img = np.array(img).reshape(28,28,1)
cv2.imwrite("./out.png",img)
scaler_2 = MinMaxScaler()
img = scaler_2.fit_transform(img.reshape(-1,1))
img = np.array(img).reshape(1,28,28,1)
print(img.shape)
result_8 = model.predict(img)
getResult(result_8[0])
# image warnanya belum di inverse sbelum dipredict 

(784, 1)
(1, 28, 28, 1)


5

In [119]:
# model.save("model.h5")
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(model, "public/")
print(tfjs.__version__)

3.13.0
